In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# SO(3) group operations and dynamics

In [ ]:
from src.groups.SON import *
G = SON(3)
print(G)

# visualization
from src.plotting import *

In [ ]:
# setup for testing different versions of dynamics
q = tensor(np.array([1e-6,0,0]))
g = G.psif(q)
v = tensor(np.array([0,1,1]))

In [ ]:
def initialize(G):
    """ add left-/right-invariant metric related structures to group --- sub-Riemannian case """
    self = G

    g = G.sym_element()
    hatxi = G.sym_Vvector() # \RR^G_dim vector
    q = G.sym_Vvector() # element in psi coordinates    
    vg = G.sym_vector() # \RR^{NxN} tangent vector at g
    wg = G.sym_vector() # \RR^{NxN} tangent vector at g
    pg = G.sym_covector() # \RR^{NxN} cotangent vector at g
    xiv = G.sym_LAvector()
    xiw = G.sym_LAvector()
    w = G.sym_coordsvector() # \RR^G_dim tangent vector in coordinates
    v = G.sym_coordsvector() # \RR^G_dim tangent vector in coordinates
    mu = G.sym_Vcovector() # \RR^G_dim LA cotangent vector in coordinates
    
    G.W = theano.shared(np.eye(G.dim.eval(),G.dim.eval())) # considered a map LA^* -> LA
    def cogV(cov=None,cow=None): # cometric in V
        if not cov and not cow:
            return G.W
        elif cov and not cow:
            return T.tensordot(G.W,cov,(1,0))
        elif cov.type == T.vector().type and cow.type == T.vector().type:
            return T.dot(cov,T.dot(G.W,cow))
        elif cov.type == T.matrix().type and cow.type == T.matrix().type:
            return T.tensordot(cov,T.tensordot(G.W,cow,(1,0)),(0,0))
        else:
            assert(False)
    G.cogV = cogV
    def cogLA(coxiv): # cometric in Lie algebra LA
        cov = G.LAtoV(coxiv)
        return G.VtoLA(G.cogV(cov))
    G.cogLA = cogLA
    def cogpsi(hatxi,p=None):
        g = G.psi(hatxi)
        dinvpsi = T.nlinalg.MatrixPinv()(G.dpsi(hatxi).reshape((G.N*G.N,G.dim))).reshape((G.dim,G.N,G.N))
        if p:
            coxiv = G.invcopb(g,T.tensordot(dinvpsi,p,(0,0)))
            xiv = G.cogLA(coxiv)        
            return T.tensordot(dinvpsi,G.invpf(g,xiv),((1,2),(0,1)))
        else:
            xiv = G.cogLA(G.invcopb(g,dinvpsi.dimshuffle((1,2,0))))        
            return T.tensordot(dinvpsi,G.invpf(g,xiv),((1,2),(0,1)))

    G.cogpsi = cogpsi
    p = G.sym_coordscovector() # \RR^G_dim cotangent vector in coordinates
    G.cogpsi_evf = theano.function([hatxi,p],G.cogpsi(hatxi,p))
    G.cogpsif = theano.function([hatxi],G.cogpsi(hatxi))

    """ group Hamiltonian from invariant metric --- sub-Riemannian case """
    # Hamiltonian using psi map
    def Hpsi(q,p):
        return .5*T.dot(p.flatten(),G.cogpsi(q,p).flatten())
    G.Hpsi = Hpsi
    G.Hpsif = theano.function([q,p],G.Hpsi(q,p))

    # default Hamiltonian
    G.H = G.Hpsi
    G.Hf = theano.function([q,p],G.H(q,p))

In [ ]:
from src.group import invariant_metric
invariant_metric.initialize(G)
initialize(G)
p = G.sharppsif(q,v)
mu = G.sharpVf(v)
print(p)
print(mu)

# from src.group import energy
# energy.initialize(G)

In [ ]:
print(q,p)
print(G.Hf(q,p))
print(G.cogpsi_evf(q,p))
print(G.cogpsif(q))

In [ ]:
# Hamiltonian dynamics
from src.dynamics import Hamiltonian
Hamiltonian.initialize(G,use_charts=False)

# test Hamiltionian and gradients
print(p)
print(G.Hf(q,p))

# geodesic
qsv = G.Exp_Hamiltoniantf(q,p)
gsv = np.array([G.psif(q) for q in qsv])
newfig()
G.plot_path(gsv)
plt.show()
(ts,qpsv) = G.Hamiltonian_dynamicsf(q,p)
psv = qpsv[:,1,:]
print("Energy: ",np.array([G.Hf(q,p) for (q,p) in zip(qsv,psv)]))